## Word2Vec model

In [3]:
import numpy as np # linear algebra
import os # accessing directory structure
import json
import glob

In [4]:
#################### Custom functions to find norm of a matrix and perform some cosine distance similarity ############################
import numpy as np
def find_norm(syn0):
    syn0norm = (syn0 / np.sqrt((syn0 ** 2).sum(-1))[..., np.newaxis]).astype(np.float32)
    return syn0norm


def argsort(x, topn=None, reverse=False):
    """
    Return indices of the `topn` smallest elements in array `x`, in ascending order.
    If reverse is True, return the greatest elements instead, in descending order.
    """
    x = np.asarray(x)  # unify code path for when `x` is not a numpy array (list, tuple...)
    if topn is None:
        topn = x.size
    if topn <= 0:
        return []
    if reverse:
        x = -x
    if topn >= x.size or not hasattr(np, 'argpartition'):
        return np.argsort(x)[:topn]
    # numpy >= 1.8 has a fast partial argsort, use that!
    most_extreme = np.argpartition(x, topn)[:topn]
    return most_extreme.take(np.argsort(x.take(most_extreme)))  # resort topn into order

def find_similar(des_norm, vec_norm):
    dists = np.dot(des_norm, vec_norm)

    best = argsort(dists, reverse=True)

    dist_sort = np.sort(dists)[::-1]

    return dist_sort, best


def similar(word, Vocab, idx2Vocab, MAT, topN=20):
    if word in Vocab:
        index = Vocab[word]
        query_vector = MAT[index]
        probs, indexes = find_similar(MAT, query_vector)

        words = [idx2Vocab[i] for i in indexes[:topN]]
        words_probs = dict(zip(words, probs[:topN]))

        return sorted(words_probs.items(), key=lambda x: x[1], reverse=True)
    else:
        return "OOV"
    
    
def similar_cross_matrix(query_vector , MAT , idx2Vocab , topN=20):

    probs, indexes = find_similar(MAT, query_vector)

    words = [idx2Vocab[i] for i in indexes[:topN]]
    words_probs = dict(zip(words, probs[:topN]))

    return sorted(words_probs.items(), key=lambda x: x[1], reverse=True) 

In [35]:
import json

emebddings = np.load('word2vec_bi_gram.syn0.npy') #### find_norm helps us to normalize embeddings, to simplify cosine distance to np.dot()
vocab      = json.load(open('word2vec_bi_gram.vocab.json'))
vocab_reverse = {k:v for v, k in vocab.items()}
#vocab = json.load("word2vec_bi_gram.vocab.json")
#emebddings = "OneDrive/Documents/1University/4 Bergen/TEL/SLATE/word2vec.ipynb"

AttributeError: 'str' object has no attribute 'read'

'C:\\Users\\wirze\\OneDrive\\Documents\\1University\\4 Bergen\\TEL\\SLATE'

In [33]:
#vocab.reverse()
#vocab_reverse = json.dumps(vocab)
vocab

<_io.TextIOWrapper name='word2vec_bi_gram.vocab.json' mode='r' encoding='cp1252'>

In [43]:
similar("sleep disorder" , vocab , vocab_reverse , emebddings)

[('disorders such', 21.214859),
 ('sleep apnea', 19.710213),
 ('anxiety disorder', 19.690964),
 ('bipolar disorder', 19.16166),
 ('disorders and', 19.138802),
 ('depressive disorder', 19.073448),
 ('anxiety disorders', 18.81599),
 ('abdominal pain', 18.751617),
 ('major depressive', 18.65441),
 ('symptoms such', 18.630896),
 ('sleep disorders', 18.558748),
 ('mental disorders', 18.421566),
 ('disorder or', 18.401009),
 ('personality disorder', 18.295654),
 ('nausea and', 18.293915),
 ('disorders including', 18.285454),
 ('nausea vomiting', 18.218536),
 ('deficit hyperactivity', 18.211435),
 ('syndrome is', 18.172626),
 ('sleep disorder', 18.161236)]